In [1]:
import re
import time

import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
url = "https://www.spotrac.com/nba/transactions/"
idx, player, position, trade_date, trade_details = [], [], [], [], []

In [ ]:
options = webdriver.ChromeOptions()
custom_user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
options.add_argument(f"user-agent={custom_user_agent}")
# options.add_argument("--headless")
path = "/usr/local/bin/chromedriver"
chrome_service = webdriver.ChromeService(executable_path=path)
driver = webdriver.Chrome(service=chrome_service, options=options)

try:
    driver.get(url)
    time.sleep(2)

    dropdown_element = driver.find_element(By.ID, "year")
    select = Select(dropdown_element)

    select.select_by_visible_text("2024")

    time.sleep(2)

    li_elements = driver.find_elements(By.CSS_SELECTOR, "li.list-group-item.d-flex.list-group-item-secondary")

    for index, li_element in enumerate(li_elements):
        try:

            player_link = li_element.find_element(By.TAG_NAME, "a")
            player_text = player_link.text.strip()

            position_match = re.search(r'\((.*?)\)', player_text)

            if position_match:
                pos = position_match.group(1)
                name = player_text.replace(f" ({pos})", "").strip()
            else:
                name = player_text
                pos = "Unknown"

            print(name, pos, end="\r")

            small_element = li_element.find_element(By.CSS_SELECTOR, "small.d-block")
            full_text = small_element.text.strip()

            date_element = small_element.find_element(By.TAG_NAME, "strong")
            date = date_element.text.strip()
            
            details = full_text.replace(date, "").strip().lstrip("- ")

            idx.append(index)
            player.append(name)
            position.append(pos)
            trade_date.append(date)
            trade_details.append(details)

        except:
            pass

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()

In [4]:
for i in [idx, player, position, trade_date, trade_details]:
    print(len(i))

807
807
807
807
807


In [5]:
df = pd.DataFrame({
    "player": player,
    "position": position,
    "trade_date": trade_date,
    "trade_details": trade_details
})

In [6]:
df

,player,position,trade_date,trade_details
0,Fred VanVleet,PG,"Dec 31, 2024","Fined $35,000 for making reckless contact with..."
1,Tyler Herro,SG,"Dec 31, 2024","Fined $25,000 for role in on-court altercation..."
2,Terry Rozier,PG,"Dec 31, 2024",Suspended 1 game for involvement in on-court a...
3,Tyler Herro,SG,"Dec 29, 2024","Fined $2,000 for ejection from MIA-HOU game"
4,Amen Thompson,PG,"Dec 29, 2024","Fined $2,000 for ejection from MIA-HOU game"
...,...,...,...,...
802,Alex Fudge,F,"Jan 06, 2024",Waived by Los Angeles (LAL)
803,Armoni Brooks,SG,"Jan 06, 2024",Waived by Brooklyn (BKN)
804,Skylar Mays,SG,"Jan 06, 2024","Waived by Portland (POR); leaves behind $850,0..."
805,Ricky Rubio,PG,"Jan 04, 2024",Waived by Cleveland (CLE) - agreed to buyout


In [7]:
df.to_csv("trades_2024.csv")